In [1]:
import requests 
import urllib
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint
from shapely import wkt
import shapely.speedups
from shapely.ops import transform, nearest_points
import plotly.express as px
from pyproj import crs
import plotly.graph_objects as go
import os
import gdal
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import glob
from functools import partial
import pyproj
import osmnx as ox
from IPython.display import Image
import scipy.ndimage as ndimage
from functools import reduce
### below files are local python programs. Make sure to paste them wherever you're running this notebook
import polygonize as pz
##
import geocoder
from pandana.loaders import osm
import pandana
import pylab as pl
ox.config(log_console=True, use_cache=True)
pl.rcParams["figure.figsize"] = (10,10)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_city_proj_crs(to_crs, val=0):
    
    """
    Function to indentify local projection for cities dynamically
    
    Input:
    to_crs : name of city / country; epsg if known
    
    Returns:
    Local epsg (in string)
    """

    if isinstance(to_crs, int):
        to_crs = to_crs
    elif isinstance(to_crs, str):
        city, country = to_crs.split(',')
        url = "http://epsg.io/?q={}&format=json&trans=1&callback=jsonpFunction".format(city)
        r = requests.get(url)
        if r.status_code == 200:
            js = json.loads(r.text[14:-1])
            
            if js['number_result'] != 0:
                lis = []
                for i in js['results']:
                    res = i
                    if (res['unit'] == 'metre') and (res['accuracy'] == 1.0):
                        lis.append(res['code'])
                if len(lis) == 0:
                    for i in js['results']:
                        res = i
                        if res['unit'] == 'metre':
                            lis.append(res['code'])
                    return lis[val]
                else:
                    return lis[val]
   
            else:
                if country.strip() == 'United Kingdom of Great Britain and Northern Ireland':
                    country = 'United Kingdom'
                elif country.strip() == 'Venezuela (Bolivarian Republic of)':
                    country = 'Venezuela'
                elif country.strip() == 'Viet Nam':
                    country = 'Vietnam'
                
                url = "http://epsg.io/?q={}&format=json&trans=1&callback=jsonpFunction".format(country)
                r = requests.get(url)
                if r.status_code == 200:
                    js = json.loads(r.text[14:-1])

                    if js['number_result'] != 0:
                        lis = []
                        for i in js['results']:
                            res = i
                            if (res['unit'] == 'metre') and (res['accuracy'] == 1.0):
                                lis.append(res['code'])
                        if len(lis) == 0:
                            for i in js['results']:
                                res = i
                                if res['unit'] == 'metre':
                                    lis.append(res['code'])
                            return lis[val]
                        else:
                            return lis[val]  

In [3]:
def convert_geom_to_shp(shapely_polygon, city, out_crs=None):
    string = city.split(',')[0]
    
    df = pd.DataFrame(
    {'City': [string],
     'geometry': [wkt.dumps(shapely_polygon)]})
    
    df['geometry'] = df['geometry'].apply(wkt.loads)
    
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    if out_crs:
        gdf.crs = {'init' : 'epsg:{}'.format(out_crs)}

        gdf.to_crs(epsg=4326, inplace=True)
    else:
        gdf.crs = {'init' : 'epsg:{}'.format(4326)}
    
    return gdf

In [4]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio accepts them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [5]:
def get_iso(city):
    """
    Function to get ISO-3 codes for countries
    
    Input:
    city: city name (Ideally in (city, country) format)
    
    Returns:
    ISO-3 code for the country
    """
    
    try:
        country = city.split(',')[1].strip().lower()
        if country == 'south korea':  ### incorrect output for South Korea's ISO code with API
            return 'kor'
        elif country == 'india':
            return 'ind'
        elif country == 'iran':
            return 'irn'
        else:
            url = "https://restcountries.eu/rest/v2/name/{}".format(country)
            r = requests.get(url)
            if len(r.json())>1 :
                for i in range(len(r.json())):
                    if country in r.json()[i]['name'].lower():
                        return r.json()[i]['alpha3Code'].lower()
            else:
                return r.json()[0]['alpha3Code'].lower()
    except IndexError:
        url = "https://restcountries.eu/rest/v2/capital/{}".format(city)
        r = requests.get(url)
        return r.json()[0]['alpha3Code'].lower()

In [6]:
def factors(n):    
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [7]:
### Performs sliding window cumulative pop_tfa estimation for each pixel
def test_func(values):
    #print (values)
    return values.sum()


x = np.array([[1,2,3],[4,5,6],[7,8,9]])

footprint = np.array([[1,1,1],
                      [1,1,1],
                      [1,1,1]])

In [54]:
def polygonize_raster(ras_path, shp_path, string):
    """
    Function to polygonize a raster based on the pixel size of base raster.
    
    Inputs:
    ras_path: path to base raster location that is to be polygonized
    shp_path: path to where the shapefile will be saved
    string: name of the city
    
    Returns:
    Geodataframe with polygons equivalent to raster pixels.
    """
    
    print("Polygonizing Raster!!")
    
    import polygonize as pz
    
    outSHPfn = shp_path
    lat, lon = pz.main(ras_path,outSHPfn)

    sh = gpd.read_file(shp_path)

    rio = rasterio.open(ras_path)
    
    sh.crs = rio.meta['crs']
    
    shp_arr = np.array(sh.geometry).reshape(rio.shape[0], rio.shape[1])
    pols = []
    for row in range(shp_arr.shape[0]-1):
        for col in range(shp_arr.shape[1]-1):
            pols.append(shapely.geometry.box(shp_arr[row+1][col].x, shp_arr[row+1][col].y, shp_arr[row][col+1].x, shp_arr[row][col+1].y ))

    gdf = gpd.GeoDataFrame()
    gdf['ID'] = [i for i in range(len(pols))]
    gdf['geometry'] = pols
    gdf.set_geometry('geometry', inplace=True)
    #gdf.crs = {'init':'epsg:4326'}
    gdf.crs = rio.meta['crs']
    print("Populating avearge height!!")

    av_h = []
    for i in gdf.geometry:
        coords = getFeatures(convert_geom_to_shp(i, string))
        out_img, out_transform = mask(dataset=rio, shapes=coords, crop=True)
        av_h.append(out_img.sum()/out_img.shape[2])

    gdf['avg_height'] = av_h
    gdf.to_crs(epsg=4326, inplace=True)
    gdf['Lon'] = [i.centroid.x for i in gdf.geometry]
    gdf['Lat'] = [i.centroid.y for i in gdf.geometry]

    
    return gdf

In [10]:
def get_population(city, gdf, wp = True, fb = None):
    
    ## Assiging Facebook population to pixels 
    iso = get_iso(city)
    
    wp_pop = []
    if fb:
        pop = rasterio.open(r"C:\Users\wb542830\OneDrive - WBG\Facebook\population_{}_2018-10-01.tif".format(iso))
    else:
        pop = rasterio.open(r"M:\Gaurav\GPSUR\Data\WorldPop_2019\{}_ppp_2019.tif".format(iso))
    for i in gdf.index:
        _gdf = gdf[gdf.index == i]
        #_gdf.to_crs(pop.meta['crs'], inplace=True)

        _coords = getFeatures(_gdf)
        try:
            _out_img, _out_transform = mask(dataset=pop, shapes=_coords, crop=True)

            outimg = np.nan_to_num(_out_img)
            outimg = outimg.reshape(outimg.shape[1], outimg.shape[2])
            wp_pop.append(outimg.sum())
        except ValueError:
            wp_pop.append(0)
            
    return wp_pop

In [55]:
def get_hotspots(city):
    string = city.split(',')[0]
    
    ### DLR raster file
    dest_path = r"M:\Gaurav\GPSUR\Data\DLR Data\{}_WSF3D_AW3D30.tif".format(string)
    ras = rasterio.open(dest_path)
    
    ## path to shapefile that will be editted 
    shp_path = r'C:\Users\wb542830\OneDrive - WBG\GPSUR\COVID\shapefiles\{}_ghsl_clip.shp'.format(string)
    
    ## Polygonize raster converts raster into polygon
    gdf  = polygonize_raster(dest_path, shp_path, string)
    
    out_proj = get_city_proj_crs(city)
    
    gdf_copy = gdf.to_crs(epsg= int(out_proj) )
    
    gdf['pixel_area'] = [i.area for i in gdf_copy.geometry]
    
    gdf['tfa'] = [(gdf.avg_height[i] * gdf.pixel_area[i]) / 3 for i in gdf.index]
    
    pop = get_population(city, gdf, wp = True)
    
    if len(pop) < 2:
        pop = get_population(city, gdf, wp = False, fb = True)
        
    gdf['pop_2019'] = pop
    gdf['pop_2019'] = [i if i>0 else 0 for i in gdf.pop_2019]
    
    gdf['pop_tfa'] = [gdf.pop_2019[i] / gdf.tfa[i] for i in gdf.index]
    
    gdf['pop_tfa'] = [0 if pd.isna(i) else i for i in gdf.pop_tfa]
    gdf['pop_tfa'] = [0 if i == np.inf else i for i in gdf.pop_tfa]
    gdf['pop_tfa'] = [0 if i == -np.inf else i for i in gdf.pop_tfa]
    
    fac = list(factors(len(gdf)))
    a = fac[int(len(fac)/2)]
    b = int(len(gdf) / a)

    results = ndimage.generic_filter(np.array(gdf.pop_tfa).reshape(a,b), test_func, footprint=footprint)

    gdf['poptfa_all'] = results.flatten()
    
    return gdf
   

In [12]:
## %%time
gdf = get_hotspots("Manila, Philippines")

Polygonizing Raster!!
0 of 1076 rows processed
100 of 1076 rows processed
200 of 1076 rows processed
300 of 1076 rows processed
400 of 1076 rows processed
500 of 1076 rows processed
600 of 1076 rows processed
700 of 1076 rows processed
800 of 1076 rows processed
900 of 1076 rows processed
1000 of 1076 rows processed
Populating avearge height!!


### Access to Services

Steps below have to repeated based on different types of services.

In [ ]:
x = ox.gdf_from_place("Manila, Philippines", which_result=2).geometry.iloc[0].envelope.bounds

In [58]:
def osm_pois_data(bounds, service):
    
    if service == 'toilets':
        amenities = ['toilets', 'washroom', 'restroom']
        osm_tags = '"amenity"~"{}"'.format('|'.join(amenities))
    elif service == 'water_points':
        amenities = ['water_points', 'drinking_water', 'pumps', 'water_pumps', 'well']
        osm_tags = '"amenity"~"{}"'.format('|'.join(amenities))
    elif service == 'shops':
        amenities = ['supermarket', 'convenience', 'general', 'department_stores', 'wholesale', 'grocery', 'general']
        osm_tags = '"shop"~"{}"'.format('|'.join(amenities))
    else:
        osm_tags = None
        
    pois = osm.node_query(bounds[1], bounds[0], bounds[3], bounds[2],tags=osm_tags)    
    
    pois['geometry'] = (list(zip(pois.lon,pois.lat)))
    pois['geometry'] = pois.geometry.apply(lambda x: Point(x))
    pois = gpd.GeoDataFrame(pois, geometry='geometry')
    pois.crs = {'init':'epsg:4326'}

    return pois

In [ ]:
pois = (x, 'water_points')

In [ ]:
out_crs = int(get_city_proj_crs("Manila, Philippines"))
pois_crs(epsg=out_crs, inplace=True)
dest = MultiPoint([i for i in pois.geometry])

In [ ]:
gdf_copy = gdf.to_crs(epsg=out_crs) ## Converting gdf to local projection

In [ ]:
dist = []
for i in gdf_copy.index:
    if i % 10000 == 0:
        print("{0} of {1} rows processed" .format(i, len(gdf_copy)))

    temp_cent = gdf_copy.geometry.iloc[i].centroid

    nearest_geoms = nearest_points(temp_cent)
    dist.append(nearest_geoms[0].distance(nearest_geoms[1]))

In [ ]:
gdf['dis_water'] = dist

In [ ]:
gdf['weight'] = [1 / math.sqrt(i) if i > 70 else 1  for i in gdf.dis_water ]

In [ ]:
## Adjusting for 'transit pixels'
pop_weight = (gdf.pop_tfa * gdf.weight) / 8

In [ ]:
weight_slide = ndimage.generic_filter(np.array(pop_weight).reshape(276,252), test_func, footprint=footprint)

In [ ]:
gdf['service_tfa'] = weight_slide.flatten()

In [ ]:
## Each pixel's density + service pixel density
gdf['pixel_density'] = gdf.poptfa_all + gdf.service_tfa

In [ ]:
gdf.to_file("Manila_Hotspots.shp")